### Segmentation Inference using saved model on TILES ###

In [ ]:
from fastai.basics import *
from fastai.vision import *
from fastai.vision.core import *
from fastai.vision.data import *
from fastai.vision import models
from fastai.vision.all import *
from fastai.metrics import *
from fastai.data.all import *
from fastai.callback import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from albumentations import ShiftScaleRotate, CoarseDropout, Cutout
from albumentations import Compose

In [ ]:
model_name = 'deeplabv3+_resnet50_Aug_25_20x_bin_1e-05' # best binary model
# model_name = 'hrnet_hrnet_w30_Aug_25_20x_1e-05' # best multiclass model

In [ ]:
class TargetMaskConvertTransformBinary(ItemTransform):
    def __init__(self):
        pass
    def encodes(self, x):
        img,mask = x

        #Convert to array
        mask = np.array(mask)

        # Change 255 for 1
        mask[mask == 29] = 1
        mask[mask == 71] = 1
        mask[mask == 99] = 1
        mask[mask == 106] = 1
        mask[mask == 111] = 1
        mask[mask == 118] = 1
        mask[mask == 146] = 1
        mask[mask == 154] = 1
        mask[mask == 158] = 1
        mask[mask == 172] = 1
        mask[mask == 178] = 1
        mask[mask == 195] = 1
        mask[mask == 212] = 1
        mask[mask == 223] = 1
        mask[mask == 233] = 1
        mask[mask == 237] = 1

        # Back to PILMask
        mask = PILMask.create(mask)


        return img, mask

In [ ]:
class SegmentationAlbumentationsTransform(ItemTransform):
    split_idx = 0
    def __init__(self, aug): self.aug = aug
    def encodes(self, x):
        img,mask = x
        aug = self.aug(image=np.array(img), mask=np.array(mask))
        return PILImage.create(aug["image"]), PILMask.create(aug["mask"])

In [ ]:
def PredsToMasksBinary(mask):
    #Dataset v2
    mask[mask == 1] = 249

    return(np.uint8(mask)) # returns np.uint8 array that will be used to create a .png mask

### Specify where predicted Masks wil be saved: ###

In [ ]:
# save predictions as masks
def SavePredsMasks(preds, idx):
    for i, pred in enumerate(preds[1]):
        img_f = foll_test['img'][i+idx]

        pred_arg = pred.argmax(dim=0).numpy()
        pred_conv = PredsToMasksBinary(pred_arg) # convert to QuPath luminosity values

        # save only masks with predictions
        if pred_conv.any() == True:
            pred_mask = Image.fromarray(pred_conv, 'L')
            mask_f = img_f.replace('.jpg', '.png')
            save_dir = '/media/14TB/aarlova_ovarian/ovarian demo/demo_segm_preds/'

            mask_f = mask_f.replace(str(Path(mask_f).parent), save_dir+(Path(mask_f).parent.stem))

            mask_f = Path(mask_f)
            mask_f.parent.mkdir(parents=True, exist_ok=True)
            new_name = str(mask_f.parent/mask_f.name)

            pred_mask.save(new_name)

In [ ]:
## custom accuracy metric (excludes void code aka 'Background')
# def acc_follicle(input, target):
#     target = target.squeeze(1)
#     mask = target != void_code
#     a = TensorImage(target[mask])
#     return (input.argmax(dim=1)[mask]==a).float().mean()

### Import packages and functions required for inference ###

In [ ]:
# ###### import for 'AI predicted masks to JSON' (3/14 version)
import ovarian_utils
from ovarian_utils import MetaPolygon
from ovarian_utils import write_qupath_noIDs
from ovarian_utils import get_AI_regions
from pathlib import Path
import numpy as np
from shapely.ops import unary_union
from collections.abc import Iterable



### Load FastAI learner and model ###

In [ ]:
learn = load_learner('/media/14TB/aarlova_ovarian/segmentation/training_log/' + model_name + '.pkl')

/home/aarlova/anaconda3/envs/pytorch_lym_clone/lib/python3.9/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='mean' instead.
  warnings.warn(warning.format(ret))


### Recreated dataset and dataloader ###

In [ ]:
#the dataset, and dls (needed for inference)
foll = pd.read_csv('/media/14TB/aarlova_ovarian/20x_norm_dgx.csv')
segdata = DataBlock(blocks=(ImageBlock),splitter=ColSplitter(),get_x=ColReader('img'),batch_tfms=[Normalize.from_stats(*imagenet_stats), IntToFloatTensor(div_mask = 1)])
dls=segdata.dataloaders(foll,bs=6)

### Example of Inference on One Slide ###

In [ ]:
imgs = ['17133893']

dir_list = ['/raid/aarlova/2000slides/tiles20x/'+i for i in imgs]
print('These tile directories will be processed:')
for d in dir_list:
    print(d)

These tile directories will be processed:
/raid/aarlova/2000slides/tiles20x/17133893


In [ ]:
# ####### Label Dictionary for Binary masks
label_dict = {249: 'Follicle', 1: 'Follicle'}
region_colors = {'Follicle': -3348737}

mask_dir = Path('/media/14TB/aarlova_ovarian/ovarian demo/demo_segm_preds/')
json_save_dir = '/media/14TB/aarlova_ovarian/ovarian demo/demo_segm_preds/'

In [ ]:
# create a test dataset from list of paths to test tiles and put into a simple DataFrame
failed = []

for d in dir_list:
    try:

        dir = Path(d)
        print('Segmentation Inference on tile dir', dir)
        paths = []
        for file in dir.iterdir():
            if file.suffix == '.jpg':
                paths.append(str(file.absolute()))

        df = pd.DataFrame()
        df['img'] = paths

        df.head()

        foll_test_ds = df


        print('Length of this dataset is',len(foll_test_ds))
        print(foll_test_ds.head)

        bs = 250
        n_batches = int(np.ceil(len(foll_test_ds)/bs))
        print('Number of batches in test dataset', n_batches)

        # inference loop
        current = 0

        for i in range(n_batches):
            # get a chunk of data from test dataset
            print('batch #', i, 'out of ', n_batches)
            foll_test = foll_test_ds[current:current+bs]
            print('length of batched test set', len(foll_test))

            # inference
            test_dl = dls.test_dl(foll_test, with_labels = False)
            preds = learn.get_preds(dl=test_dl, with_input = True)
            SavePredsMasks(preds, current)

            current += bs

        print('Done with Inference!')

        ############### if no masks were predicted, create an empty directory ###############
        save_dir = Path('/media/14TB/aarlova_ovarian/ovarian demo/demo_segm_preds/' + dir.stem)
        save_dir.mkdir(parents=True, exist_ok=True)



        ############### convert masks to json ###############
        min_area = 5000

        json_save = json_save_dir + str(dir.name) + '.json'
        print('Creating JSON at', json_save)

        AI_slide_polygons = get_AI_regions(Path(mask_dir/dir.name))
        AI_slide_labels = np.unique([p.label for p in AI_slide_polygons])

        regions = []
        region_labels = []

        for label in AI_slide_labels:
            print('Detected class', label)
            if label == 'Antral':
                buffer = 1.5
                current_label_polygons = [p.buffer(buffer) for p in AI_slide_polygons if (p.label == label and p.area > 50000)]
            else:
                buffer = 1.5
                # find all polys with this label and unary_union them
                current_label_polygons = [p.buffer(buffer) for p in AI_slide_polygons if (p.label == label and p.area > min_area)]
            print('detected before union',len(current_label_polygons))

            # unary_union on polygons of the same label
            if len(current_label_polygons) > 1:
                current_label_polygons_union = unary_union(current_label_polygons)

                if not isinstance(current_label_polygons_union, Iterable):
                    current_label_polygons_union = [current_label_polygons_union]

                current_label_polygons_union_with_label = [MetaPolygon(label, p) for p in current_label_polygons_union]

                # convert to regions and regions_labels
                for p in current_label_polygons_union_with_label:
                    new_ext_x = np.array(p.exterior.coords.xy[0])
                    new_ext_y = np.array(p.exterior.coords.xy[1])

                    new_coords = list(zip(new_ext_x, new_ext_y))
                    new_label = p.label
                    regions.append(new_coords)
                    region_labels.append(new_label)
                print('detected after union', len(current_label_polygons_union_with_label))

            elif len(current_label_polygons) == 0:
                pass

            else:
                new_ext_x = np.array(current_label_polygons[0].exterior.coords.xy[0])
                new_ext_y = np.array(current_label_polygons[0].exterior.coords.xy[1])

                new_coords = list(zip(new_ext_x, new_ext_y))
                new_label = label
                regions.append(new_coords)
                region_labels.append(new_label)

        ids = [i for i in range(len(regions))]
        write_qupath_noIDs(json_save, regions,region_labels,region_colors)
        print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

    except:
        failed.append(d)

print('Failed to infer:', len(failed), failed)

Segmentation Inference on tile dir /raid/aarlova/2000slides/tiles20x/17133893
Length of this dataset is 2028
<bound method NDFrame.head of                                                                                            img
0     /raid/aarlova/2000slides/tiles20x/17133893/17133893 [x=105464,y=37196,w=1000,h=1000].jpg
1     /raid/aarlova/2000slides/tiles20x/17133893/17133893 [x=104564,y=41696,w=1000,h=1000].jpg
2     /raid/aarlova/2000slides/tiles20x/17133893/17133893 [x=138916,y=45796,w=1000,h=1000].jpg
3      /raid/aarlova/2000slides/tiles20x/17133893/17133893 [x=20584,y=33604,w=1000,h=1000].jpg
4     /raid/aarlova/2000slides/tiles20x/17133893/17133893 [x=181536,y=15332,w=1000,h=1000].jpg
...                                                                                        ...
2023   /raid/aarlova/2000slides/tiles20x/17133893/17133893 [x=21484,y=40804,w=1000,h=1000].jpg
2024   /raid/aarlova/2000slides/tiles20x/17133893/17133893 [x=61624,y=12940,w=1000,h=1000].jpg
2025  

batch # 1 out of  9
length of batched test set 250


batch # 2 out of  9
length of batched test set 250


batch # 3 out of  9
length of batched test set 250


batch # 4 out of  9
length of batched test set 250


...

Done with Inference!
Creating JSON at /media/14TB/aarlova_ovarian/ovarian demo/demo_segm_preds/17133893.json
This folder has predictions


/media/14TB/aarlova_ovarian/ovarian demo/ovarian_utils.py:23: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  self.label = label
/media/14TB/aarlova_ovarian/ovarian demo/ovarian_utils.py:24: ShapelyDeprecationWarning: Setting custom attributes on geometry objects is deprecated, and will raise an AttributeError in Shapely 2.0
  self.id = id


done with slide
Detected class Follicle
detected before union 545


<ipython-input-19-da90a9536e9e>:83: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  current_label_polygons_union_with_label = [MetaPolygon(label, p) for p in current_label_polygons_union]


detected after union 391
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Failed to infer: 0 []
